In [11]:
flight_name = 'HALO-20240811a'
flight_date = flight_date = flight_name[5:9]+'-'+flight_name[9:11]+'-'+flight_name[11:13]
print (flight_date)

2024-08-11


In [3]:
"Generate image with flight track superimposed on GOES visible image"""

import orcestra 
from orcestra.flightplan import  LatLon, path_preview
from orcestra.weathermaps import goes_overlay
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import xarray as xr

sns.set_context("talk")

flight_date = flight_name[5:9]+'-'+flight_name[9:11]+'-'+flight_name[11:13]
tracks = xr.open_dataset('/Volumes/ORCESTRA/'+flight_name+'/bahamas/QL_'+flight_name+'_BAHAMAS_V01.nc')
path   = LatLon(lat=tracks['IRS_LAT'], lon=tracks['IRS_LON'], label=flight_name)

fig, ax = plt.subplots(
    figsize=(15, 8),
    facecolor='white',
    subplot_kw  ={"projection": ccrs.PlateCarree()}
)

path_preview(path,ax=ax,show_waypoints=False,color='#FF5349')

# goes_overlay can take some time depending on the network.  The downloaded image is cached, but the first time
# it can take 15 min to download.  Thereafter the fetching and plotting of the cached data takes about 1 min
#
goes_overlay(flight_date+'T14:15',ax)
fig.savefig(flight_name+'-track.jpeg',bbox_inches='tight')

/Users/m219063/micromamba/envs/orcestra_book/lib/python3.12/site-packages/goes2go/data.py:665: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  within=pd.to_timedelta(config["nearesttime"].get("within", "1h")),
/Users/m219063/micromamba/envs/orcestra_book/lib/python3.12/site-packages/goes2go/NEW.py:188: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  within=pd.to_timedelta(config["nearesttime"].get("within", "1h")),


In [23]:
import orcestra.sat
from datetime import datetime
import xarray as xr
import orcestra.postprocess
import orcestra.postprocess.level0
import orcestra.postprocess.level1
import matplotlib.pyplot as plt

flight_time = datetime(int(flight_name[5:9]), int(flight_name[9:11]), int(flight_name[11:13]), 12, 0, 0)
ec_track = orcestra.sat.SattrackLoader("EARTHCARE", "2024-08-10", kind="PRE").get_track_for_day(f"{flight_time:%Y-%m-%d}")
ec_track = ec_track.sel(time=slice(f"{flight_time:%Y-%m-%d} 06:00", None))

halo = xr.open_dataset('/Volumes/ORCESTRA/'+flight_name+'/bahamas/QL_'+flight_name+'_BAHAMAS_V01.nc').pipe(orcestra.postprocess.level0.bahamas).sel(time=slice(ec_track.time[0],ec_track.time[-1]))
ec   = ec_track.interp(time=halo.time)
sig_ec = 360*60*1852/(90*600)/2.
az12, az21, dist = orcestra.flightplan.geod.inv(ec.lon,ec.lat,halo.IRS_LON,halo.IRS_LAT)
print (f'Minimum distance between EarthCARE and HALO {dist.min():6.1f} +/- {sig_ec} m at {halo.time[dist.argmin()].astype(str):.21s} s')

fig, ax = plt.figure(figsize = (12, 8))

plt.plot(halo.time,dist)
ax.set_yscale('log')

#plt.savefig(f'{flight_index}_track.png')

Minimum distance between EarthCARE and HALO 1163.2 +/- 370.4 m at 2024-08-11T15:51:53.8 s


TypeError: cannot unpack non-iterable Figure object

<Figure size 1200x800 with 0 Axes>

In [13]:
import numpy as np
flight_time = np.datetime64(f'{flight_date}T12:00.00.0', 'ns')

/var/folders/_9/hj56_qws3tx_n_y2gn3mqnxr0000gn/T/ipykernel_36360/3125201585.py:2: UserWarning: no explicit representation of timezones available for np.datetime64
  xx = np.datetime64(f'{flight_date}T12:00.00.0', 'ns')


ValueError: Error parsing datetime string "2024-08-11T12:00.00.0" at position 16

In [8]:
print(flight_time)

NameError: name 'flight_time' is not defined

In [ ]:
import numpy as np
x = halo_track.where(halo_track.time > ec_track.time[ 0], drop=True)
ht = x.where(x.time < ec_track.time[ -1], drop=True)
print (ht.time)
print (ec_track.time)


In [ ]:
from scipy.interpolate import interp1d

# Original data
x_original = ec_track.time
y_original = ec_track.lat

# New indices
x_new = ht.time

# Create an interpolation function
interpolator = interp1d(x_original, y_original, kind='linear', fill_value='extrapolate')

# Interpolate to new indices
#y_new = interpolator(x_new)

print('first',x_original[0:3])

print(x_new[0:3])

In [ ]:
import orcestra.postprocess
import orcestra.postprocess.level0
import orcestra.postprocess.level1

halo = xr.open_dataset('/Volumes/ORCESTRA/'+flight_name+'/bahamas/QL_'+flight_name+'_BAHAMAS_V01.nc').pipe(orcestra.postprocess.level0.bahamas).sel(time=slice(ec_track.time[0],ec_track.time[-1]))
ec   = ec_track.interp(time=halo.time)
sig_ec = 360*60*1852/(90*600)/2.
az12, az21, dist = orcestra.flightplan.geod.inv(ec.lon,ec.lat,halo.IRS_LON,halo.IRS_LAT)
print (f'Minimum distance between EarthCARE and HALO {dist.min():6.1f} +/- {sig_ec} m at {halo.time[dist.argmin()].astype(str):.21s} s')

In [ ]:
#dist = np.sqrt((ec.lat-halo.IRS_LAT)**2 + (ec.lon-halo.IRS_LON)* np.cos(np.deg2rad(ec.lat+halo.IRS_LAT)/2)**2)*60*1.852



In [ ]:
plt.plot(halo.time,dist)
print (dist.min())

In [ ]:

print (f"EarthCARE underpass: {halo.time[dist.argmin()].astype(str):.21s} s")